In [53]:
import pandas as pd
import seaborn as sb

import pylab

import random
import math

In [54]:
class DataProcessing:
    def __init__(self):
        pass
    
    def shuffling(self, data_list):
        n = len(data_list)-1

        for i in range(n,-1,-1):
            index = random.randint(0,i)
            tmp = data_list.loc[i]
            data_list.loc[i] = data_list.loc[index]
            data_list.loc[index] = tmp
    
    def normalize(self, data_list):
        for col in data_list.columns:
            if type(data_list[col].loc[0]) is not type("text"):
                min1 = float("inf")
                max1 = float("-inf")

                for x in data_list[col]:
                    min1 = min(min1,x)
                    max1 = max(max1,x)
                    
                for i in range(len(data_list[col])):
                    data_list.at[i, col] -= min1  
                    data_list.at[i, col] /= (max1 - min1)

    def train_test_split(self, data_list):
        train_len = round((len(data_list)-1) * 0.7)
        test_len = round((len(data_list)-1) * 0.3)
        
        train = data_list[0:train_len]
        test = data_list[train_len:len(data_list)]

        test = test.reset_index(drop=True) #resetowanie indeksacji dataframeu

        return train, test
    
    def label_split(self, data_list):
        feature_list = []
        label_list = []

        sum = 0
        for col in data_list.columns:
            if type(data_list[col].loc[0]) is not type("text"):
                sum +=1
        
        for i in range(len(data_list)):
            feature_list.append(data_list.loc[i].to_list()[:sum])

        for i in range(len(data_list)):
            label_list.append(data_list.loc[i].to_list()[sum:])
        
        return feature_list, label_list

In [55]:
def gaussInator(x, mi, sigma):
    
    return (1 / (sigma * math.sqrt(2 * math.pi))) * math.exp(-((x - mi) ** 2) / (2 * sigma ** 2))

In [56]:
def naiveBayes(d_list, t_data): # dla kazdej cechy dla kazdego gatunky srednia i odchylenie, 12 srednich i odchylen, dla sprawdzanego podstawiamy wzor, iloczyn wektorowy

    dtp = DataProcessing()
    cat_avg_dict = {} # dict na średnie z podziałem kategoria {cechy}
    cat_elem_len = {} # dict na ilość elementów dla każdej cechy
    cat_std_dict = {} # dict na odchylenia

    inator_dict = {} # dict na wyliczone wartości gęstości dla każdej cechy

    product_per_cat = {} # dict z kategoriami i iloczynami cech dla każdej kategorii

    feature_list, label_list = dtp.label_split(d_list)

    sum = 0
    for col in d_list.columns:
        if type(d_list[col].loc[0]) is not type("text"): # ilość kolumn które opisują cechy nie kategorie
            sum +=1

    for label in label_list: # ustawianie dict'ów aby były podzielone na kategorie
        cat_avg_dict[label[0]] = {}
        cat_elem_len[label[0]] = {}
        cat_std_dict[label[0]] = {}
        inator_dict[label[0]] = {}
        product_per_cat[label[0]] = 1
        for col in d_list.columns[:sum]: # ustawianie dict'ów aby był podział na cechy dla każdej kategorii
            cat_avg_dict[label[0]][col] = 0
            cat_elem_len[label[0]][col] = 0
            cat_std_dict[label[0]][col] = 0
            inator_dict[label[0]][col] = 1
    

    for i, label in enumerate(label_list): # obliczanie sumy i ilości elementów dla każdej cechy
        if d_list.loc[i]['variety'] is label[0]:
            for col in d_list.columns[:sum]:
                    cat_avg_dict[label[0]][col] += d_list.loc[i][col]
                    cat_elem_len[label[0]][col] += 1

    for cat in cat_avg_dict: # obliczanie średniej przez dzielenie dla każdej cechy
        for elem in cat_avg_dict[cat]:
            cat_avg_dict[cat][elem] /= cat_elem_len[cat][elem]

    for i, label in enumerate(label_list): # obliczanie sumy odchylenia standardowego
        if d_list.loc[i]['variety'] is label[0]:
            for col in d_list.columns[:sum]:
                    cat_std_dict[label[0]][col] += (d_list.loc[i][col] - cat_avg_dict[label[0]][col])**2

    for cat in cat_avg_dict:
        for elem in cat_avg_dict[cat]: # obliczanie iloczynu i pierwiastka i mamy odchylenie
             cat_std_dict[cat][elem] /= cat_elem_len[cat][elem]
             cat_std_dict[cat][elem] = math.sqrt(cat_std_dict[cat][elem])

    for cat in cat_avg_dict: # wartości gęstości dla testowanego rekordu
        for i, elem in enumerate(cat_avg_dict[cat]):
            inator_dict[cat][elem] = gaussInator(t_data[i], cat_avg_dict[cat][elem], cat_std_dict[cat][elem])

    for cat in cat_avg_dict:
        for elem in cat_avg_dict[cat]: # końcowy iloraz wszystkich cech dla kategorii
            product_per_cat[cat] *= inator_dict[cat][elem]

    max_cat = ""
    tmp = float("-inf")

    for cat in product_per_cat: # wybieranie kategorii z maksymalnym ilorazem
        if product_per_cat[cat] >= tmp:
             tmp = product_per_cat[cat]
             max_cat = cat

    return max_cat


In [57]:
data = pd.read_csv("iris.csv")

In [58]:
#print(len(data)) # dlugosc datasetu
#print(data.loc[5]) # uzyskiwanie danych o jednym rekordzie
#print(data.loc[5].to_list()) # wypisywanie rekordu w postaci listy

#sb.pairplot(data, hue="variety") # wyswietlanie wykresow kazdej cechy
#sb.violinplot(data, x="sepal.width", y="variety", inner="quartile")
#print(data.describe()) # wypisuje uśrednione info o datasecie
#print(data)

#x = []
#for i in range(0,100,1):
#    x.append(i/10)

#y = []
#for e in x:
#    y.append(gaussInator(e,5,1))

#pylab.plot(x, y)
#pylab.title('Wykres f(x) = a*x - b')
#pylab.grid(True)
#pylab.show()


In [59]:
dtp = DataProcessing()

dtp.shuffling(data)
dtp.normalize(data)
train_data, test_data = dtp.train_test_split(data)

#print(train_data)
#print(test_data)

In [60]:
#print(naiveBayes(train_data, (6.4, 3.1, 5.5, 1.8))) # virginica bez normalizacji
#train_data.describe()

In [61]:

all=0
good=0
bad=0

for i in range(len(test_data)-1):

    result = naiveBayes(train_data, test_data.loc[i].to_list()[:4])

    if result == test_data.loc[i].to_list()[4:][0]:
        good += 1
    else:
        bad += 1

    all += 1

print("Naiwny klasyfikator Bayesa")
print(f"All: {all}, good: {good}, bad: {bad}")
print(f"Test statistic: {round(good/all,4)*100}%")

Naiwny klasyfikator Bayesa
All: 45, good: 44, bad: 1
Test statistic: 97.78%
